# Scraping Websites for Data and Cleaning
So one of my interests is korean pop music (currently taking the world by storm), and I think there are some interesting trend that can be observed so the following are some simple visualization of kpop data. This involves some new concepts I have never tried before in terms of acquiring data. This notebook will include my scraping of data from the gaon charts. 

In [ ]:
import seaborn as sns
import pandas as pd
import re
import os

In [ ]:
## need to scrape data from: gaon, spotify, 
import requests
url = "http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL"
page = requests.get(url)

In [ ]:
import bs4
soup = bs4.BeautifulSoup(page.content, 'lxml')
table = soup.find(name='table')
table

In [ ]:
result = pd.DataFrame([[td.text for td in row.findAll('td')] for row in table.findChildren('tr')])
#print(table.findChildren(['tr']))
result.head() 

In [ ]:
pt = r'\W+'
col_names = [re.sub(pt, '', th.string) for th in table.find('tr')]
result = result.drop([1,2,4,6,7], axis=1)

In [ ]:
col_names

In [ ]:
col_dict = dict(zip(result.columns, col_names[1:3]+[col_names[4]]))
result = result.rename(columns=col_dict)
result = result.drop(0)
result

In [ ]:
# clean text within the table 
cleaned_result = result.apply(lambda x: x.apply(lambda y: re.sub(r'\n', ' ', y)))
alter = result['TitleArtist'].apply(lambda x: re.sub(r'\n', 'newline ', x))

cleaned_result['Album'] = result['TitleArtist'].apply(lambda x: re.search(r'\|(.*)$', x)[1])
cleaned_result['Artist'] = result['TitleArtist'].apply(lambda x: re.search(r'(.*)\|', x)[1])
cleaned_result['Title'] = alter.apply(lambda x: re.search(r'(?<=newline).*?(?=newline)', x)[0])
cleaned_result['Week'] = 31
cleaned_result['Year'] = 2019
cleaned_result.head()

In [ ]:
# @source a web scraping article I found on Towards Data Science
# https://towardsdatascience.com/a-short-practical-how-to-guide-to-scrape-data-from-a-website-using-python-888373227d4f

PATH = os.path.join("Users","catherinegee","Desktop","cs misc","ds-for-fun") # you need to change to your local path
res = pd.DataFrame()

end = '&targetTime=31&hitYear=2019&termGbn=week'
groups = ['&targetTime=', '&hitYear=', '&termGbn=week']

opts = soup.find(name='select', attrs={'id':'chart_week_select'})
lst = []
for c in opts.children:
    lst += [c]
    
num_times = len(lst) - 2
pth = '/Users/catherinegee/Desktop/cs misc/ds-for-fun/Kpop/data/table.csv'
cleaned_result.to_csv(pth)


In [ ]:
def clean_df(result, table, week, year):
    """Cleans the table as done in example dataframe"""
    pt = r'\W+'
    col_names = [re.sub(pt, '', th.string) for th in table.find('tr')]
    if year <= 2017:
        result = result.drop([1,2,5,6], axis=1)
        col_dict = dict(zip(result.columns, col_names[1:4]))
    else:
        result = result.drop([1,2,4,6,7], axis=1)
        col_dict = dict(zip(result.columns, col_names[1:3]+[col_names[4]]))
    result = result.rename(columns=col_dict)
    result = result.drop(0)
    cleaned_result = result.apply(lambda x: x.apply(lambda y: re.sub(r'\n', ' ', y)))
    alter = result['TitleArtist'].apply(lambda x: re.sub(r'\n', 'newline ', x))

    cleaned_result['Album'] = result['TitleArtist'].apply(lambda x: re.search(r'\|(.*)$', x)[1])
    cleaned_result['Artist'] = result['TitleArtist'].apply(lambda x: re.search(r'(.*)\|', x)[1])
    cleaned_result['Title'] = alter.apply(lambda x: re.search(r'(?<=newline).*?(?=newline)', x)[0])
    cleaned_result['Week'] = week
    cleaned_result['Year'] = year
    return cleaned_result
    
def table_to_df(t, week, year): 
    """Turns the table into a datafram using pandas"""
    result = pd.DataFrame([[td.text for td in row.findAll('td')] for row in t.findChildren('tr')])
    return clean_df(result, t, week, year)

def update_date(week, year):
    """Updates the date by week number and year"""
    year = int(year)
    week = int(week) - 1
    if week == 0:
        year -= 1
        week = 52
    elif week < 10:
        week = '0' + str(week)
    #year = str(year)
    return int(week), year, groups[0] + str(week) + groups[1] + str(year) + groups[2]


In [ ]:
for i in range(num_times):
    #print(i)
    week_n = re.search(r'(?<=&targetTime=).*?(?=&hitYear=)', end)[0] 
    year = re.search(r'(?<=&hitYear=).*?(?=&termGbn=week)', end)[0] 
    w, y, end = update_date(week_n, year)
    print(w,y,end)
    url_week = url + end
    pg = requests.get(url_week)
    sp = bs4.BeautifulSoup(pg.content, 'lxml')
    table = sp.find(name='table')
    df1 = table_to_df(table, w, y)
    res = res.append(df1)
    if i % 20 == 0:
        res.to_csv(pth)

In [ ]:
test = url + '&targetTime=52&hitYear=2017&termGbn=week'
pg = requests.get(test)
sp = bs4.BeautifulSoup(pg.content, 'lxml')
table = sp.find(name='table')
table

In [ ]:
result = pd.DataFrame([[td.text for td in row.findAll('td')] for row in table.findChildren('tr')])
result.head()

In [ ]:
col_names = [re.sub(pt, '', th.string) for th in table.find('tr')]
col_names
